# CLIcK

In [11]:
from datasets import load_dataset
from typing import Tuple
import pandas as pd

ds = load_dataset("EunsuKim/CLIcK")
df = ds['train'].to_pandas()
df.head()

,id,paragraph,question,choices,answer
0,KIIP_economy_1,,다음은 한국 사회의 경제에 대한 문제이다.\n한국이 외환위기를 완전히 극복한 년도는...,"[1999년, 2000년, 2001년, 2002년]",2001년
1,KIIP_economy_2,,다음은 한국 사회의 경제에 대한 문제이다.\n1960년 한국의 1인당 국민총소득은 ...,"[79달러, 800달러, 8,000달러, 80,000달러]",79달러
2,KIIP_economy_3,,다음은 한국 사회의 경제에 대한 문제이다.\n한국이 외환위기를 겪은 년도는 언제인가?,"[1995년, 1996년, 1997년, 1998년]",1997년
3,KIIP_economy_4,,다음은 한국 사회의 경제에 대한 문제이다.\n한강은 어느 도시를 통과하는가?,"[부산, 대구, 인천, 서울]",서울
4,KIIP_economy_5,,다음은 한국 사회의 경제에 대한 문제이다.\n1970년대부터 한국에서 발달하기 시작...,"[농업, 경공업, 중화학 공업, 서비스 업]",중화학 공업


In [12]:
def extract_category(id_text: str) -> str:
    text_split = id_text.split('_')
    return text_split[0]


In [13]:
df['categories'] = df['id'].apply(lambda x: extract_category(x))
df['choices'] = df['choices'].apply(lambda x: [str(_) for _ in list(x)])
print(set(df['categories']))

keys = list(set(df['categories']))

{'PSAT', 'KHB', 'Kedu', 'PSE', 'KIIP', 'TK', 'CSAT'}


In [15]:
df['choices'].iloc[0]

['1999년', '2000년', '2001년', '2002년']

In [16]:
df = df[['paragraph', 'question', 'choices', 'answer', 'categories']]
dfs = df.groupby('categories')

In [17]:
dfs.get_group('TK')

,paragraph,question,choices,answer,categories
1690,,( )에 들어갈 가장 알맞은 것을 고르십시오.\n내일 친구와 함께 놀이공원에 ( ).,"[가는 편이다, 가는 중이다, 가기로 했다, 간 적이 있다]",가기로 했다,TK
1691,,( )에 들어갈 가장 알맞은 것을 고르십시오.\n혼자 살다 보니 시간이 ( ) 가족...,"[지나거나, 지나도록, 지나거든, 지날수록]",지날수록,TK
1692,,다음 괄호 안 부분과 의미가 비슷한 것을 고르십시오.\n아침에 늦게 (일어나는 바람...,"[일어난 탓에, 일어난 김에, 일어나는 대신, 일어나는 대로]",일어난 탓에,TK
1693,,다음 괄호 안 부분과 의미가 비슷한 것을 고르십시오.\n경기에서 이기고 지는 것은 ...,"[연습할 따름이다, 연습할 모양이다, 연습하기 나름이다, 연습하기 십상이다]",연습하기 나름이다,TK
1694,,( )에 들어갈 가장 알맞은 것을 고르십시오.\n해가 뜨는 것을 ( ) 아침 일...,"[보아야, 보려고, 보거나, 보는데]",보려고,TK
...,...,...,...,...,...
1990,올해 서른두 살인 준은 어렸을 때부터 특출나게 뛰어난 재능이 없다는 점이 큰 불만이...,다음 글의 내용으로 알 수 있는 것을 고르십시오.,"[준은 여행지에서 해 뜨는 방향을 한 번에 찾았다., 준의 부모님은 아들과 같은 능...",준은 여행지에서 해 뜨는 방향을 한 번에 찾았다.,TK
1991,서양 역사에서는 15ㆍ16세기를 ‘위대한 발견의 시대’라고 부른다. 콜럼버스 처럼 ...,( )에 들어갈 말로 가장 알맞은 것을 고르십시오.,"[값비싼 물건을 매매했기, 다양한 지도를 수집했기, 미지의 땅을 찾아 떠났기, 탐험...",미지의 땅을 찾아 떠났기,TK
1992,서양 역사에서는 15ㆍ16세기를 ‘위대한 발견의 시대’라고 부른다. 콜럼버스 처럼 ...,다음 글의 주제로 가장 알맞은 것을 고르십시오.,"[콜럼버스는 새로운 대륙을 개척하는 데 기여한 바가 크다., 신대륙 발견은 동방의 ...",유럽인들의 탐험은 항로를 만들어 세계를 연결시켰다는 데 의의가 있다.,TK
1993,"세계는 신에너지, 자동화, 우주여행 등이 주도하는 시대로 급속히 접어들고 있다. 세...",다음 글에 나타난 필자의 태도로 가장 알맞은 것을 고르십시오.,"[과학 정책에 대한 정부의 지나친 개입을 경계하고 있다., 과학 기술 발전을 위해서...",과학 정책에 대한 정부의 지나친 개입을 경계하고 있다.,TK


In [ ]:
# save each group to folder with name: category_test.csv
folder = "/work/u5110390/BenchWeaver/evaluation_data/click/data/test"
import os
for key in keys:
    group = dfs.get_group(key)
    group.to_csv(os.path.join(folder, f"{key}_test.csv"), index=False)

# HAE_RAE_BENCH 1.0

In [44]:
from datasets import load_dataset, get_dataset_config_names
from typing import Tuple
import pandas as pd


In [45]:
opqa = ['lyrics_denoising', 'proverbs_denoising']
mcqa = ['correct_definition_matching', 'csat_geo', 'csat_law', 'csat_socio', 'date_understanding', 'general_knowledge', 'history', 'loan_words', 'rare_words', 'standard_nomenclature', 'reading_comprehension']

## MCQA

In [46]:
# check options for each task
import ast
from tqdm.auto import  tqdm
answers = []
with  tqdm(total=len(mcqa)) as pbar:
    for task in mcqa:
        pbar.set_postfix_str(task)
        ds = load_dataset("HAERAE-HUB/HAE_RAE_BENCH_1.1", task)
        df = ds['test'].to_pandas()
        answers.extend(df['answer'].tolist())
        for idx in range(df.shape[0]):
            try:
                option_list = ast.literal_eval(df['options'].iloc[idx])
            except:
                option_list = df['options'].iloc[idx].split("|")
            if len(option_list) != 5:
                print(idx, df['options'].iloc[idx])
        pbar.update(1)
print(set(answers))
print(len(set(answers)))

100%|██████████| 11/11 [00:17<00:00,  1.61s/it, reading_comprehension]     

{'(D)', '(E)', '(A)', '(B)', '(C)'}
5


In [47]:
# check question format
with  tqdm(total=len(mcqa)) as pbar:
    for task in mcqa:
        pbar.set_postfix_str(task)
        ds = load_dataset("HAERAE-HUB/HAE_RAE_BENCH_1.1", task)
        df = ds['test'].to_pandas()
        print(f"-------------------{task}-------------------")
        print(df['query'].iloc[0])
        print("=================================")
        pbar.update(1)

  9%|▉         | 1/11 [00:04<00:41,  4.11s/it, csat_geo]                   

-------------------correct_definition_matching-------------------
다음 문장에서 "가노"가 사용된 의미로 옳은 것을 고르시요. 
    
### 문장: 
이미 타계한 지 오래인 그의 조부가 서유하 씨 부친이었던 대지주 서 참봉 댁 가노로 있었다.

### 선택지: 
(A) 일본 중부 지방의 중앙부에 있는 현. 제사업, 정밀 기계, 전기 기계 따위의 산업이 발달하였으며, 쌀 및 고랭지 채소의 산지로 알려져 있다. 사과, 배, 포도 따위의 과일도 많이 난다. 현청 소재지는 나가노, 면적은 1만 3633㎢.
(B) 16세기에서 17세기 사이의 제정 러시아의 부호 집안. 16세기 중반 이반 사세(Ivan四世)의 신임을 얻으면서부터 융성하여 광대한 직할령, 제염, 제철 따위의 특권을 부여받고 사병(私兵)ㆍ요새의 소유도 인정받았으며 시베리아의 개척에 공헌하였다.
(C) 이탈리아의 무용가ㆍ안무가(1769~1821). 발레를 춤추는 드라마로 끌어올린 혁신적인 안무가였다. 베토벤이 그를 위하여 작곡한 <프로메테의 창조물>의 안무가 유명하다.
(D) ‘가내 노비’를 줄여 이르는 말.
(E) 일본 나가노현 북부에 있는 시. 인쇄, 식료품, 전기 기기 따위의 공업과 상업이 발달하였다. 교외에서는 사과, 야채 따위의 재배가 활발하다. 현청 소재지이다.

### 정답:


 18%|█▊        | 2/11 [00:05<00:21,  2.44s/it, csat_law]

-------------------csat_geo-------------------
다음 자료는 대동여지도 제작에 관한 가상 대화 장면이다. (ㄱ)~(ㄹ)에 대한 옳은 설명만을 <보기>에서 있는 대로 고른 것은?

### 참고:
<word start>(ㄱ)산줄기<word end>는 끊어지지 않게 하고, 물줄기는 <word start>(ㄴ)쌍선<word end>과 단선으로 구분해서 새기도록 하여라.
예. 도로에는 <word start>(ㄷ)방점<word end>도 표시하겠습니다. 그런데 <word start>(ㄹ)목판<word end>에 새기자니 시간이 많이 걸리네요.

<보기>
(A). (ㄱ)은 하천 유역을 나누는 경계가 된다.
(B). (ㄴ)은 배가 다닐 수 있는 하천을 나타낸 것이다.
(C). (ㄷ)은 교통 및 통신 시설을 표현한 기호이다.
(D). (ㄹ)로 제작되어 지도의 대량 생산이 가능하다.
### 선택지: (A)  (A), (B)  (B)  (C), (D)  (C)  (A), (B), (C)  (D)  (A), (B), (D)  (E)  (B), (C), (D)


 27%|██▋       | 3/11 [00:06<00:15,  1.93s/it, csat_socio]

...

-------------------rare_words-------------------
다음 질문을 읽고 정답으로 가장 알맞은 것을 고르시요.
    
### 질문: 
물건을 만들 때에 다듬지 않고 거칠게 대강 만드는 일의 의미를 가진 단어로 알맞은 것은?

### 선택지: 
(A) 건널목
(B) 나들목
(C) 속재목
(D) 건목
(E) 조건문

### 정답:


 91%|█████████ | 10/11 [00:19<00:01,  1.51s/it, reading_comprehension]

-------------------standard_nomenclature-------------------
다음 질문을 읽고 정답으로 가장 알맞은 것을 고르시요.
    
### 질문: 
욜드의 올바른 표준 전문 용어로 알맞은 것은?

### 선택지: 
(A) 청노년
(B) 청소원
(C) 청바지
(D) 청동끌
(E) 청소년복

### 정답:


100%|██████████| 11/11 [00:20<00:00,  1.85s/it, reading_comprehension]

-------------------reading_comprehension-------------------
다음 지문을 읽고 질문에 대답으로 가장 적절한 것을 고르시요.

### 지문: 
이제 한 편의 이야기를 들려 드립니다. 잘 듣고 물음에 답하십시오.
자, 여러분! 안녕하십니까? 오늘은 제가 어제 꾼 꿈 이야기 하날 들려 드리겠습니다. 전 꿈속에서 낯선 거리를 걷고 있었습니다. 그러다가 홍미로운 간판을 발견했답니다. 행 복을 파는 가게. 그렇게 쓰여 있었습니다. 전 호기심으로 문을 열고 들어갔답니다. 그곳 에서는 한 노인이 물건을 팔고 있었습니다. 전 잠시 머뭇거리다가 노인에게 다가가서 물 었습니다. 여기서는 무슨 물건을 파느냐고요. 노인은 미소를 지으며, 원하는 것은 뭐든 다 살 수 있다고 말했습니다. 저는 제 귀를 의심했습니다. '무엇이든 다?' 전 무엇을 사야 할까 생각하다가 말했답니다. "사랑, 부귀 그리고 지혜하고 건강도 사고 싶습니다. 저 자신뿐 아니라 우리 가족 모두 를 위해서요. 지금 바로 살 수 있나요?" 그러자 노인은 빙긋이 웃으며 대답했습니다. "젊은이, 한번 잘 보게나. 여기에서 팔고 있는 것은 무르익은 과일이 아니라 씨앗이라 네. 앞으로 좋은 열매를 맺으려면 이 씨앗들을 잘 가꾸어야 할 걸세."
    
### 질문:
 이 이야기에서 얻을 수 있는 교훈으로 가장 적절한 것은?

### 선택지:
(A) 새로운 세계에 대한 열망을 가져야 한다.
(B) 주어진 기회를 능동적으로 활용해야 한다.
(C) 큰 것을 얻으려면 작은 것은 버려야 한다.
(D) 물질적 가치보다 정신적 가치를 중시해야 한다.
(E) 소망하는 바를 성취하기 위해서는 노력을 해야 한다.

### 정답:


In [48]:
# redefine the task
import re
from typing import List

def parse_options(options_list_text: str)->List[str]:
    try:
        option_list = ast.literal_eval(options_list_text)
    except:
        option_list = options_list_text.split("|")
    return option_list

def clean_answer(answers: str)->str:
    return re.sub(r"\(|\)", "", answers)


def process_mcqa_task(task) -> pd.DataFrame:
    ds = load_dataset("HAERAE-HUB/HAE_RAE_BENCH_1.1", task)
    df = ds['test'].to_pandas()
    
    # Apply parse_options and expand into separate columns
    options_df = df['options'].apply(lambda x: pd.Series(parse_options(x), index=["A", "B", "C", "D", "E"]))
    df = pd.concat([df, options_df], axis=1)

    df['answer'] = df['answer'].apply(clean_answer)
    df['question'] = df['query']
    df['categories'] = "mcqa"
    return df[['question', 'A', 'B', 'C', 'D', 'E', 'answer', 'categories']]

In [ ]:
with tqdm(total=len(mcqa)) as pbar:
    for task in mcqa:
        pbar.set_postfix_str(task)
        df = process_mcqa_task(task)
        df.to_csv(f"/work/u5110390/BenchWeaver/evaluation_data/hae-rae-bench/data/test/{task}_test.csv", index=False)
        pbar.update(1)

100%|██████████| 11/11 [00:19<00:00,  1.75s/it, reading_comprehension]     


## OPQA

In [51]:
with tqdm(total=len(opqa)) as pbar:
    for task in opqa:
        pbar.set_postfix_str(task)
        ds = load_dataset("HAERAE-HUB/HAE_RAE_BENCH_1.1", task)
        df = ds['test'].to_pandas()
        print(f"-------------------{task}-------------------")
        print(df['query'].iloc[0])
        print("=================================")
        print(df['options'].iloc[0])
        print("=================================")
        print(df['answer'].iloc[0])
        pbar.update(1)

 50%|█████     | 1/2 [00:01<00:01,  1.74s/it, proverbs_denoising]

-------------------lyrics_denoising-------------------
다음은 제이티엘(JTL)의 노래 Without Your Love의 가사 중 일부분입니다. 주어진 단어를 활용해 [MASK]에 들어갈 가사를 생성하시요.

### 가사: 
it's without without your love
[MASK]'[MASK] [MASK] [MASK]'[MASK] [MASK] [MASK] [MASK]
believe it you'd better believe it
감정의 흐름을 잃어버려 공간 속에 난 갇혔어
without your loving
햇빛을 내려 비가 내리게 
햇님을 담아 별님을 담아 나 상상했던 이상을 찾아 
It's without without your love

### 단어: ['love', 'without', "It's", 'your', "it's", 'without']

## 정답:
nan
It's without it's without your love


100%|██████████| 2/2 [00:03<00:00,  1.58s/it, proverbs_denoising]

-------------------proverbs_denoising-------------------
다음은 어떤 한국 속담에 대한 뜻풀이 입니다. 다음 뜻풀이를 읽고 주어진 단어를 사용해 해당 속담을 생성하시요.

### 뜻풀이: 
일의 차례가 뒤바뀌었음을 비유적으로 이르는 말.

### 단어: ['뒤에', '하고', '말', '앞에', '뒤에', '말', '할', '할', '앞에', '하고']

## 정답:
nan
앞에 할 말 뒤에 하고 뒤에 할 말 앞에 하고


In [52]:

def process_opqa_task(task) -> pd.DataFrame:
    ds = load_dataset("HAERAE-HUB/HAE_RAE_BENCH_1.1", task)
    df = ds['test'].to_pandas()
    df['A'] = ""
    df['B'] = ""
    df['C'] = ""
    df['D'] = ""
    df['E'] = ""
    df['question'] = df['query']
    df['categories'] = "opqa"
    
    return df[['question', 'A', 'B', 'C', 'D', 'E', 'answer', 'categories']]

In [ ]:
with tqdm(total=len(opqa)) as pbar:
    for task in opqa:
        pbar.set_postfix_str(task)
        df = process_opqa_task(task)
        df.to_csv(f"/work/u5110390/BenchWeaver/evaluation_data/hae-rae-bench/data/test/{task}_test.csv", index=False)
        pbar.update(1)

100%|██████████| 2/2 [00:02<00:00,  1.36s/it, proverbs_denoising]


# KMMLU (Hard)

In [1]:
supercategories = {
        "accounting": "HUMSS",
        "agricultural_sciences": "Other",
        "aviation_engineering_and_maintenance": "Applied Science",
        "biology": "STEM",
        "chemical_engineering": "STEM",
        "chemistry": "STEM",
        "civil_engineering": "STEM",
        "computer_science": "STEM",
        "construction": "Other",
        "criminal_law": "HUMSS",
        "ecology": "STEM",
        "economics": "HUMSS",
        "education": "HUMSS",
        "electrical_engineering": "STEM",
        "electronics_engineering": "Applied Science",
        "energy_management": "Applied Science",
        "environmental_science": "Applied Science",
        "fashion": "Other",
        "food_processing": "Other",
        "gas_technology_and_engineering": "Applied Science",
        "geomatics": "Applied Science",
        "health": "Other",
        "industrial_engineer": "Applied Science",
        "information_technology": "STEM",
        "interior_architecture_and_design": "Other",
        "law": "HUMSS",
        "machine_design_and_manufacturing": "Applied Science",
        "management": "HUMSS",
        "maritime_engineering": "Applied Science",
        "marketing": "Other",
        "materials_engineering": "STEM",
        "mechanical_engineering": "STEM",
        "nondestructive_testing": "Applied Science",
        "patent": "Other",
        "political_science_and_sociology": "HUMSS",
        "psychology": "HUMSS",
        "public_safety": "Other",
        "railway_and_automotive_engineering": "Applied Science",
        "real_estate": "Other",
        "refrigerating_machinery": "Other",
        "social_welfare": "HUMSS",
        "taxation": "HUMSS",
        "telecommunications_and_wireless_technology": "Applied Science",
        "korean_history": "HUMSS",
        "math": "STEM"
    }

In [2]:
set(supercategories.values())

{'Applied Science', 'HUMSS', 'Other', 'STEM'}

In [3]:
import json

mapping = {}
for k, v in supercategories.items():
    mapping[k] = {
        "name": k.replace("_", " "),
        "category": v
    }
    
with open("/work/u5110390/BenchWeaver/evaluation_data/kmmlu/mapping.json", "w") as f:
    json.dump(mapping, f, indent=2)
    
with open("/work/u5110390/BenchWeaver/evaluation_data/kmmlu-hard/mapping.json", "w") as f:
    json.dump(mapping, f, indent=2)

In [4]:
task = ['Accounting', 'Agricultural-Sciences', 'Aviation-Engineering-and-Maintenance', 'Biology', 'Chemical-Engineering', 'Chemistry', 'Civil-Engineering', 'Computer-Science', 'Construction', 'Criminal-Law', 'Ecology', 'Economics', 'Education', 'Electrical-Engineering', 'Electronics-Engineering', 'Energy-Management', 'Environmental-Science', 'Fashion', 'Food-Processing', 'Gas-Technology-and-Engineering', 'Geomatics', 'Health', 'Industrial-Engineer', 'Information-Technology', 'Interior-Architecture-and-Design', 'Law', 'Machine-Design-and-Manufacturing', 'Management', 'Maritime-Engineering', 'Marketing', 'Materials-Engineering', 'Mechanical-Engineering', 'Nondestructive-Testing', 'Patent', 'Political-Science-and-Sociology', 'Psychology', 'Public-Safety', 'Railway-and-Automotive-Engineering', 'Real-Estate', 'Refrigerating-Machinery', 'Social-Welfare', 'Taxation', 'Telecommunications-and-Wireless-Technology', 'Korean-History', 'Math']

task_list = [t.lower().replace("-", "_") for t in task]
task_list

['accounting',
 'agricultural_sciences',
 'aviation_engineering_and_maintenance',
 'biology',
 'chemical_engineering',
 'chemistry',
 'civil_engineering',
 'computer_science',
 'construction',
 'criminal_law',
 'ecology',
 'economics',
 'education',
 'electrical_engineering',
 'electronics_engineering',
 'energy_management',
 'environmental_science',
 'fashion',
 'food_processing',
 'gas_technology_and_engineering',
 'geomatics',
 'health',
 'industrial_engineer',
 'information_technology',
 'interior_architecture_and_design',
 'law',
 'machine_design_and_manufacturing',
 'management',
 'maritime_engineering',
 'marketing',
 'materials_engineering',
 'mechanical_engineering',
 'nondestructive_testing',
 'patent',
 'political_science_and_sociology',
 'psychology',
 'public_safety',
 'railway_and_automotive_engineering',
 'real_estate',
 'refrigerating_machinery',
 'social_welfare',
 'taxation',
 'telecommunications_and_wireless_technology',
 'korean_history',
 'math']

In [5]:
from datasets import load_dataset
from tqdm.auto import tqdm
import pandas as pd

In [7]:
import os
def casting_idx2option(idx) -> str:
    idx2option = { "1": "A", "2": "B", "3": "C", "4": "D", "5": "E", "6": "F"}
    return idx2option[str(idx)]

with tqdm(total=len(task)) as pbar:
    for t in task:
        pbar.set_postfix_str(t)
        ds = load_dataset("HAERAE-HUB/KMMLU", t)
        train_df = ds['train'].to_pandas()
        test_df = ds['test'].to_pandas()
        dev_df = ds['dev'].to_pandas()
        train_df['answer'] = train_df['answer'].apply(casting_idx2option)
        test_df['answer'] = test_df['answer'].apply(casting_idx2option)
        dev_df['answer'] = dev_df['answer'].apply(casting_idx2option)
        task_name = t.lower().replace("-", "_")
        os.makedirs("/work/u5110390/BenchWeaver/evaluation_data/kmmlu/data/dev", exist_ok=True)
        os.makedirs("/work/u5110390/BenchWeaver/evaluation_data/kmmlu/data/test", exist_ok=True)
        os.makedirs("/work/u5110390/BenchWeaver/evaluation_data/kmmlu/data/val", exist_ok=True)
        train_df.to_csv(f"/work/u5110390/BenchWeaver/evaluation_data/kmmlu/data/dev/{task_name}_dev.csv", encoding="utf-8", index=False)
        dev_df.to_csv(f"/work/u5110390/BenchWeaver/evaluation_data/kmmlu/data/val/{task_name}_val.csv", encoding="utf-8", index=False)
        test_df.to_csv(f"/work/u5110390/BenchWeaver/evaluation_data/kmmlu/data/test/{task_name}_test.csv", encoding="utf-8", index=False)
        pbar.update(1)

  0%|          | 0/45 [00:00<?, ?it/s]

In [24]:

        
with tqdm(total=len(task)) as pbar:
    for t in task:
        pbar.set_postfix_str(t)
        task_name = t.lower().replace("-", "_")
        ds = load_dataset("HAERAE-HUB/KMMLU-HARD", task_name)
        test_df = ds['test'].to_pandas()
        dev_df = ds['dev'].to_pandas()
        test_df['answer'] = test_df['answer'].apply(casting_idx2option)
        dev_df['answer'] = dev_df['answer'].apply(casting_idx2option)
        test_df.rename(columns={"cot": "explanation"}, inplace=True)
        dev_df.rename(columns={"cot": "explanation"}, inplace=True)
        os.makedirs("/work/u5110390/BenchWeaver/evaluation_data/kmmlu-hard/data/dev", exist_ok=True)
        os.makedirs("/work/u5110390/BenchWeaver/evaluation_data/kmmlu-hard/data/test", exist_ok=True)
        dev_df.to_csv(f"/work/u5110390/BenchWeaver/evaluation_data/kmmlu-hard/data/dev/{task_name}_dev.csv", index=False)
        test_df.to_csv(f"/work/u5110390/BenchWeaver/evaluation_data/kmmlu-hard/data/test/{task_name}_test.csv", index=False)
        pbar.update(1)

  0%|          | 0/45 [00:00<?, ?it/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/46 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

...

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/44 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

In [1]:
import pandas as pd
df = pd.read_csv("/work/u5110390/BenchWeaver/evaluation_data/kmmlu/data/test/agricultural_sciences_test.csv")

# TMMLU+

In [ ]:
from datasets import load_dataset
import os
import pandas as pd
import zipfile
import shutil

task_list = [
             'engineering_math', 'dentistry', 'traditional_chinese_medicine_clinical_medicine', 'clinical_psychology', 'technical', 'culinary_skills', 'mechanical', 'logic_reasoning', 'real_estate',
             'general_principles_of_law', 'finance_banking', 'anti_money_laundering', 'ttqav2', 'marketing_management', 'business_management', 'organic_chemistry', 'advance_chemistry',
             'physics', 'secondary_physics', 'human_behavior', 'national_protection', 'jce_humanities', 'politic_science', 'agriculture', 'official_document_management',
             'financial_analysis', 'pharmacy', 'educational_psychology', 'statistics_and_machine_learning', 'management_accounting', 'introduction_to_law', 'computer_science', 'veterinary_pathology',
             'accounting', 'fire_science', 'optometry', 'insurance_studies', 'pharmacology', 'taxation', 'trust_practice', 'geography_of_taiwan', 'physical_education', 'auditing', 'administrative_law',
             'education_(profession_level)', 'economics', 'veterinary_pharmacology', 'nautical_science', 'occupational_therapy_for_psychological_disorders',
             'basic_medical_science', 'macroeconomics', 'trade', 'chinese_language_and_literature', 'tve_design', 'junior_science_exam', 'junior_math_exam', 'junior_chinese_exam',
             'junior_social_studies', 'tve_mathematics', 'tve_chinese_language', 'tve_natural_sciences', 'junior_chemistry', 'music', 'education', 'three_principles_of_people',
             'taiwanese_hokkien'
            ]

base_dir = "/work/u5110390/BenchWeaver/evaluation_data/tmmluplus/data"
for task in task_list:
    val = load_dataset('ikala/tmmluplus', task)['validation']
    dev = load_dataset('ikala/tmmluplus', task)['train']
    test = load_dataset('ikala/tmmluplus', task)['test']
    # Convert datasets to DataFrames
    val_df = val.to_pandas()
    dev_df = dev.to_pandas()
    test_df = test.to_pandas()
    # Define file paths
    val_file = os.path.join(base_dir, 'val', f"{task}_val.csv")
    dev_file = os.path.join(base_dir, 'dev', f"{task}_dev.csv")
    test_file = os.path.join(base_dir, 'test', f"{task}_test.csv")
    # Create directories if they don't exist
    os.makedirs(os.path.dirname(val_file), exist_ok=True)
    os.makedirs(os.path.dirname(dev_file), exist_ok=True)
    os.makedirs(os.path.dirname(test_file), exist_ok=True)
    # Save DataFrames to CSV files
    val_df.to_csv(val_file, index=False)
    dev_df.to_csv(dev_file, index=False)
    test_df.to_csv(test_file, index=False)

# Define the zip file name
zip_file = "tmmluplus.zip"

# Function to zip a directory and its contents
def zip_dir(directory, zip_file):
    with zipfile.ZipFile(zip_file, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(directory):
            for file in files:
                zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), directory))

# Zip the directory
zip_dir(base_dir, zip_file)
# Remove the base directory after zipping
shutil.rmtree(base_dir)

/home/joeyliang/anaconda3/envs/BenchWeaver/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import pandas as pd
df = pd.read_csv("/work/u5110390/BenchWeaver/evaluation_data/tmmluplus/subject.tsv", delimiter="\t")
df

,subject,name,category
0,dentistry,牙醫學,health
1,traditional_chinese_medicine_clinical_medicine,中醫臨床醫學,health
2,clinical_psychology,臨床心理學,psychology
3,technical,技術工相關,other
4,culinary_skills,餐旅,other
...,...,...,...
62,music,音樂科,other
63,education,教育常識,education
64,three_principles_of_people,三民主義,culture
65,taiwanese_hokkien,閩南語,culture


In [14]:
mmlu_mapping = {
    "health": "Other",
    "psychology": "Social Sciences",
    "other": "Other",
    "law": "Social Sciences",
    "business": "Other",
    "culture": "Humanities",
    "chemistry": "STEM",
    "physics": "STEM",
    "politics": "Social Sciences",
    "philosophy": "Humanities",
    "math": "STEM",
    "biology": "STEM",
    "engineering": "STEM",
    "computer science": "STEM",
    "education": "Social Sciences",
    "economics": "Social Sciences",
    "geography": "Social Sciences"
}


In [15]:
catogories = {}
for idx in range(df.shape[0]):
    catogories[df['subject'][idx]] = {
        "name": df['name'][idx],
        "category": mmlu_mapping[df['category'][idx]]
    }

In [ ]:
import json
with open("/work/u5110390/BenchWeaver/evaluation_data/tmmluplus/mapping.json", "w") as f:
    json.dump(catogories, f, ensure_ascii=False, indent=2)

# TMLU

In [6]:
mapping ={
    "AST_civics": {"name": "分科測驗公民", "category": "Social Science"},
    "AST_geography": {"name": "分科測驗地理", "category": "Social Science"},
    "CAP_civics": {"name": "會考公民", "category": "Social Science"},
    "CAP_geography": {"name": "會考地理", "category": "Social Science"},
    "GSAT_civics": {"name": "學測公民", "category": "Social Science"},
    "GSAT_geography": {"name": "學測地理", "category": "Social Science"},
    "accountant": {"name": "會計師", "category": "Social Science"},
    "clinical_psychologist": {"name": "臨床心理師", "category": "Social Science"},
    "AST_biology": {"name": "分科測驗生物", "category": "STEM"},
    "AST_chemistry": {"name": "分科測驗化學", "category": "STEM"},
    "AST_mathematics": {"name": "分科測驗數學", "category": "STEM"},
    "AST_physics": {"name": "分科測驗物理", "category": "STEM"},
    "CAP_biology": {"name": "會考生物", "category": "STEM"},
    "CAP_chemistry": {"name": "會考化學", "category": "STEM"},
    "CAP_earth_science": {"name": "會考地球科學", "category": "STEM"},
    "CAP_mathematics": {"name": "會考數學", "category": "STEM"},
    "CAP_physics": {"name": "會考物理", "category": "STEM"},
    "GSAT_biology": {"name": "學測生物", "category": "STEM"},
    "GSAT_chemistry": {"name": "學測化學", "category": "STEM"},
    "GSAT_earth_science": {"name": "學測地球科學", "category": "STEM"},
    "GSAT_mathematics": {"name": "學測數學", "category": "STEM"},
    "GSAT_physics": {"name": "學測物理", "category": "STEM"},
    "AST_chinese": {"name": "分科測驗國文", "category": "Humanities"},
    "AST_history": {"name": "分科測驗歷史", "category": "Humanities"},
    "CAP_chinese": {"name": "會考國文", "category": "Humanities"},
    "CAP_history": {"name": "會考歷史", "category": "Humanities"},
    "GSAT_chinese": {"name": "學測國文", "category": "Humanities"},
    "GSAT_history": {"name": "學測歷史", "category": "Humanities"},
    "tour_guide": {"name": "導遊", "category": "Humanities"},
    "tour_leader": {"name": "領隊", "category": "Humanities"},
    "lawyer_qualification": {"name": "律師資格", "category": "Humanities"},
    "driving_rule": {"name": "台灣駕駛規則", "category": "Taiwan Specific"},
    "teacher_qualification": {"name": "教師資格", "category": "Taiwan Specific"},
    "taiwan_tourist_resources": {"name": "台灣觀光資源", "category": "Taiwan Specific"},
    "basic_traditional_chinese_medicine": {"name": "中醫基礎醫學", "category": "Others"},
    "clinical_traditional_chinese_medicine": {"name": "中醫針灸", "category": "Others"},
    "nutritionist": {"name": "營養師", "category": "Others"}
}


In [22]:
task_list = list(mapping.keys())
task_list

['AST_civics',
 'AST_geography',
 'CAP_civics',
 'CAP_geography',
 'GSAT_civics',
 'GSAT_geography',
 'accountant',
 'clinical_psychologist',
 'AST_biology',
 'AST_chemistry',
 'AST_mathematics',
 'AST_physics',
 'CAP_biology',
 'CAP_chemistry',
 'CAP_earth_science',
 'CAP_mathematics',
 'CAP_physics',
 'GSAT_biology',
 'GSAT_chemistry',
 'GSAT_earth_science',
 'GSAT_mathematics',
 'GSAT_physics',
 'AST_chinese',
 'AST_history',
 'CAP_chinese',
 'CAP_history',
 'GSAT_chinese',
 'GSAT_history',
 'tour_guide',
 'tour_leader',
 'lawyer_qualification',
 'driving_rule',
 'teacher_qualification',
 'taiwan_tourist_resources',
 'basic_traditional_chinese_medicine',
 'clinical_traditional_chinese_medicine',
 'nutritionist']

In [ ]:
import json
with open("/work/u5110390/BenchWeaver/evaluation_data/tmlu/mapping.json", "w") as f:
    json.dump(mapping, f, ensure_ascii=False, indent=2)

In [ ]:
from datasets import load_dataset
from tqdm.auto import tqdm
import pandas as pd
import os
import zipfile
import shutil

def process_df(df:pd.DataFrame) -> pd.DataFrame:
    choice_lists = []
    for idx in range(df.shape[0]):
        answer_opt = df['answer'].iloc[idx]
        choice_list = [
            df.loc[idx, 'A'],
            df.loc[idx, 'B'],
            df.loc[idx, 'C'],
            df.loc[idx, 'D'],
            df.loc[idx, 'E'],
            df.loc[idx, 'F'],
        ]
        df.loc[idx, 'choices'] = str(choice_list)
        df.loc[idx, 'answer'] = df.loc[idx, answer_opt]
    df = df[['question', 'choices', 'answer']]
    return df

base_dir = "/work/u5110390/BenchWeaver/evaluation_data/tmlu/data"
os.makedirs(base_dir, exist_ok=True)

for task in tqdm(mapping.keys()):
    ds = load_dataset("miulab/tmlu", task)
    dev_df = process_df(ds['dev'].to_pandas())
    test_df = process_df(ds['test'].to_pandas())
    dev_file = os.path.join(base_dir, 'dev', f"{task}_dev.csv")
    test_file = os.path.join(base_dir, 'test', f"{task}_test.csv")
    os.makedirs(os.path.dirname(dev_file), exist_ok=True)
    os.makedirs(os.path.dirname(test_file), exist_ok=True)
    dev_df.to_csv(dev_file, index=False)
    test_df.to_csv(test_file, index=False)

# Define the zip file name
zip_file = "/work/u5110390/BenchWeaver/evaluation_data/tmlu/tmlu.zip"

# Function to zip a directory and its contents
def zip_dir(directory, zip_file):
    with zipfile.ZipFile(zip_file, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(directory):
            for file in files:
                zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), directory))

# Zip the directory
zip_dir(base_dir, zip_file)
# Remove the base directory after zipping
shutil.rmtree(base_dir)

100%|██████████| 37/37 [01:17<00:00,  2.08s/it]


# DRCD

In [ ]:
import json
from typing import Any
def load_json(file_path:str) -> Any:
    with open(file_path, "r") as f:
        data = json.load(f)
    return data

train_data = load_json("/work/u5110390/BenchWeaver/evaluation_data/drcd/DRCD_training.json")
dev_data = load_json("/work/u5110390/BenchWeaver/evaluation_data/drcd/DRCD_dev.json")
test_data = load_json("/work/u5110390/BenchWeaver/evaluation_data/drcd/DRCD_test.json")

In [2]:
train_record = {
    "title": [],
    "id": [],
    "context": [],
    "question": [],
    "answer": [],
    "answer_start": [],
}
dev_record = {
    "title": [],
    "id": [],
    "context": [],
    "question": [],
    "answer": [],
    "answer_start": [],
}
test_record = {
    "title": [],
    "id": [],
    "context": [],
    "question": [],
    "answer": [],
    "answer_start": [],
}

In [3]:
from tqdm.auto import tqdm
for idx in tqdm(range(len(train_data['data']))):
    for paragraph in train_data['data'][idx]['paragraphs']:
        for qa in paragraph['qas']:
            train_record['title'].append(train_data['data'][idx]['title'])
            train_record['id'].append(qa['id'])
            train_record['context'].append(paragraph['context'])
            train_record['question'].append(qa['question'])
            train_record['answer'].append(qa['answers'][0]['text'])
            train_record['answer_start'].append(qa['answers'][0]['answer_start'])

for idx in tqdm(range(len(dev_data['data']))):
    for paragraph in dev_data['data'][idx]['paragraphs']:
        for qa in paragraph['qas']:
            dev_record['title'].append(dev_data['data'][idx]['title'])
            dev_record['id'].append(qa['id'])
            dev_record['context'].append(paragraph['context'])
            dev_record['question'].append(qa['question'])
            dev_record['answer'].append(qa['answers'][0]['text'])
            dev_record['answer_start'].append(qa['answers'][0]['answer_start'])
            
for idx in tqdm(range(len(test_data['data']))):
    for paragraph in test_data['data'][idx]['paragraphs']:
        for qa in paragraph['qas']:
            test_record['title'].append(test_data['data'][idx]['title'])
            test_record['id'].append(qa['id'])
            test_record['context'].append(paragraph['context'])
            test_record['question'].append(qa['question'])
            test_record['answer'].append(qa['answers'][0]['text'])
            test_record['answer_start'].append(qa['answers'][0]['answer_start'])

/home/joeyliang/anaconda3/envs/BenchWeaver/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 378/378 [00:00<00:00, 165116.32it/s]


In [5]:
import pandas as pd
train_df = pd.DataFrame(train_record)
dev_df = pd.DataFrame(dev_record)
test_df = pd.DataFrame(test_record)
test_df

,title,id,context,question,answer,answer_start
0,梵文,1147-2-1,要探討從梨俱吠陀到波你尼時代梵語的發展，可以考察印度教其它文本，如娑摩吠陀、夜柔吠陀、阿闥婆...,夜柔吠陀與阿闥婆吠陀均可以最為研究哪一門語言的參考？,梵語,14
1,梵文,1147-2-2,要探討從梨俱吠陀到波你尼時代梵語的發展，可以考察印度教其它文本，如娑摩吠陀、夜柔吠陀、阿闥婆...,哪一本書規範了梵語的正確語法？,八篇書,133
2,梵文,1147-2-3,要探討從梨俱吠陀到波你尼時代梵語的發展，可以考察印度教其它文本，如娑摩吠陀、夜柔吠陀、阿闥婆...,中古印度-雅利安語方言的前身與哪一門語言都同時在古印度使用？,梵語,315
3,梵文,1147-3-1,波你尼所定義的梵語是從更早的「吠陀」形式演化出來的。學者經常把吠陀梵語和古典或「波你尼」梵語...,波你尼梵語與哪一門語言非常相似？,吠陀梵語,31
4,梵文,1147-3-2,波你尼所定義的梵語是從更早的「吠陀」形式演化出來的。學者經常把吠陀梵語和古典或「波你尼」梵語...,印度教的最早宗教文本以什麼語言撰寫？,吠陀梵語,96
...,...,...,...,...,...,...
3488,喀喇汗国,6516-5-2,著名的薩圖克·博格拉汗是這個國家第一個皈依伊斯蘭教的可汗，更重要的是他也是世界歷史上第一個皈...,突厥人的第一個伊斯蘭教王朝是?,喀喇汗國,58
3489,喀喇汗国,6516-5-3,著名的薩圖克·博格拉汗是這個國家第一個皈依伊斯蘭教的可汗，更重要的是他也是世界歷史上第一個皈...,哪一個地區的穆斯林商人讓薩圖克對伊斯蘭文化十分嚮往?,阿圖什,181
3490,納瓦拉,6519-1-1,"納瓦拉，正式名稱是納瓦拉特許共同體，是西班牙北部一個自治區，全區僅有納瓦拉一省，面積10,3...",哪一個城市是納瓦拉的首府?,潘普洛納,62
3491,納瓦拉,6519-1-2,"納瓦拉，正式名稱是納瓦拉特許共同體，是西班牙北部一個自治區，全區僅有納瓦拉一省，面積10,3...",上納瓦拉在哪一年被西班牙兼併?,1512年,214


In [ ]:
train_df.to_csv("/work/u5110390/BenchWeaver/evaluation_data/drcd/data/dev/all_dev.csv", index=False)
dev_df.to_csv("/work/u5110390/BenchWeaver/evaluation_data/drcd/data/val/all_val.csv", index=False)
test_df.to_csv("/work/u5110390/BenchWeaver/evaluation_data/drcd/data/test/all_test.csv", index=False)

# awesome-taiwan-knowledge

In [ ]:
import json
from typing import Any
import pandas as pd
def load_json(file_path:str) -> Any:
    with open(file_path, "r") as f:
        data = json.load(f)
    return data

data = load_json("/work/u5110390/BenchWeaver/evaluation_data/awesome-taiwan-knowledge/TTQA_1.0.0_tw_llama_v1.0.json")
data_record = {
    "title": [],
    "question": [],
    "answer": [],
}
for data_dict in data:
    data_record['title'].append(data_dict['answer'])
    data_record['question'].append(data_dict['examples'])
    data_record['answer'].append("以下皆為正確答案：" + "或".join([f"「{ans}」" for ans in data_dict['correct_answers']]))
    
df = pd.DataFrame(data_record)
df

,title,question,answer
0,櫻花鉤吻鮭,它是位於亞熱帶的台灣內唯一一種溫帶性魚類，也是只產於台灣的特有櫻鮭亞種，為冰河孑遺生物。由於...,以下皆為正確答案：「櫻花鉤吻鮭」
1,臺灣獼猴,臺語也稱烏肢猴 ，是舊世界猴獼猴屬的的物種，分布於臺灣本島及澎湖，為常見的臺灣特有種，也是島...,以下皆為正確答案：「臺灣獼猴」
2,台灣黑熊,是臺灣特有的亞洲黑熊亞種，胸前的V字型白色條紋是亞洲黑熊共有的特徵。\n該動物的名稱是：,以下皆為正確答案：「台灣黑熊」
3,藍腹鷴,俗稱臺灣山雞，為台灣特有種。深藍色的大型雉類。雄性頭頸黑色，羽冠白色有時帶黑斑，後頸及頸側為...,以下皆為正確答案：「藍腹鷴」
4,台灣藍鵲,臺灣特有種鳥類，成鳥的喙和腳呈紅色，虹膜呈金黃色，頭至頸、胸部呈黑色，翅膀末端呈白色，身體其...,以下皆為正確答案：「台灣藍鵲」
...,...,...,...
59,"MP魔幻力量,魔幻力量",臺灣流行樂團，為台灣首組雙主唱男子樂團。它由主唱廷廷、主唱嘎嘎、DJ鼓鼓、吉他手雷堡、貝斯手...,以下皆為正確答案：「MP魔幻力量」或「魔幻力量」
60,五月天,它是於1999年出道的臺灣樂團，由主唱阿信、團長暨吉他手怪獸、吉他手石頭、貝斯手瑪莎和鼓手冠...,以下皆為正確答案：「五月天」
61,草根詩社,它於1975年5月在台北成立，並且創《草根詩刊》。該社成員有張香華、羅青、李男、詹澈、邱豐松...,以下皆為正確答案：「草根詩社」
62,十大建設,它是指中華民國政府於1974年至1979年間，時任行政院院長蔣經國為改善臺灣的基礎設施及產業...,以下皆為正確答案：「十大建設」


In [ ]:
df.to_csv("/work/u5110390/BenchWeaver/evaluation_data/awesome-taiwan-knowledge/data/test/all_test.csv", index=False)

# CMMLU

In [12]:
name_en2zh = {
    "agronomy": "农学",
    "anatomy": "解剖学",
    "ancient_chinese": "古汉语",
    "arts": "艺术学",
    "astronomy": "天文学",
    "business_ethics": "商业伦理",
    "chinese_civil_service_exam": "中国公务员考试",
    "chinese_driving_rule": "中国驾驶规则",
    "chinese_food_culture": "中国饮食文化",
    "chinese_foreign_policy": "中国外交政策",
    "chinese_history":"中国历史",
    "chinese_literature": "中国文学",
    "chinese_teacher_qualification": "中国教师资格",
    "clinical_knowledge": "临床知识",
    "college_actuarial_science":"大学精算学",
    "college_education":"大学教育学",
    "college_engineering_hydrology": "大学工程水文学",
    "college_law": "大学法律",
    "college_mathematics": "大学数学",
    "college_medical_statistics":"大学医学统计",
    "college_medicine": "大学医学",
    "computer_science": "计算机科学",
    "computer_security": "计算机安全",
    "conceptual_physics": "概念物理学",
    "construction_project_management": "建设工程管理",
    "economics": "经济学",
    "education": "教育学",
    "electrical_engineering": "电气工程",
    "elementary_chinese":"小学语文",
    "elementary_commonsense":"小学常识",
    "elementary_information_and_technology": "小学信息技术",
    "elementary_mathematics": "初等数学",
    "ethnology": "民族学",
    "food_science": "食品科学",
    "genetics": "遗传学",
    "global_facts": "全球事实",
    "high_school_biology": "高中生物",
    "high_school_chemistry": "高中化学",
    "high_school_geography": "高中地理",
    "high_school_mathematics": "高中数学",
    "high_school_physics": "高中物理学",
    "high_school_politics": "高中政治",
    "human_sexuality": "人类性行为",
    "international_law": "国际法学",
    "journalism": "新闻学",
    "jurisprudence": "法理学",
    "legal_and_moral_basis": "法律与道德基础",
    "logical": "逻辑学",
    "machine_learning": "机器学习",
    "management": "管理学",
    "marketing": "市场营销",
    "marxist_theory": "马克思主义理论",
    "modern_chinese": "现代汉语",
    "nutrition": "营养学",
    "philosophy": "哲学",
    "professional_accounting": "专业会计",
    "professional_law": "专业法学",
    "professional_medicine": "专业医学",
    "professional_psychology": "专业心理学",
    "public_relations": "公共关系",
    "security_study":"安全研究",
    "sociology": "社会学",
    "sports_science": "体育学",
    "traditional_chinese_medicine": "中医中药",
    "virology": "病毒学",
    "world_history":"世界历史",
    "world_religions": "世界宗教",
}

subcategories = {
    "agronomy": ['other'],
    "anatomy": ['biology'],
    "ancient_chinese": ['linguistics','china specific'],
    "arts": ['arts'],
    "astronomy": ['physics'],
    "business_ethics": ['business'],
    "chinese_civil_service_exam": ['politics','china specific'],
    "chinese_driving_rule": ['other','china specific'],
    "chinese_food_culture": ['culture','china specific'],
    "chinese_foreign_policy": ['politics','china specific'],
    "chinese_history":['history','china specific'],
    "chinese_literature": ['literature','china specific'],
    "chinese_teacher_qualification": ['education','china specific'],
    "college_actuarial_science":['math'],
    "college_education":['education'],
    "college_engineering_hydrology": ['engineering'],
    "college_law": ['law'],
    "college_mathematics": ['math'],
    "college_medical_statistics":['statistics'],
    "clinical_knowledge": ['other'],
    "college_medicine": ['other'],
    "computer_science": ['computer science'],
    "computer_security": ['other'],
    "conceptual_physics": ['physics'],
    "construction_project_management": ['other','china specific'],
    "economics": ['economics'],
    "education": ['education'],
    "elementary_chinese":['linguistics','china specific'],
    "elementary_commonsense":['other','china specific'],
    "elementary_information_and_technology": ['other'],
    "electrical_engineering": ['engineering'],
    "elementary_mathematics": ['math'],
    "ethnology": ['culture','china specific'],
    "food_science": ['other'],
    "genetics": ['biology'],
    "global_facts": ['global'],
    "high_school_biology": ['biology'],
    "high_school_chemistry": ['chemistry'],
    "high_school_geography": ['geography'],
    "high_school_mathematics": ['math'],
    "high_school_physics": ['physics'],
    "high_school_politics": ['politics','china specific'],
    "human_sexuality": ['other'],
    "international_law": ['law'],
    "journalism": ['sociology'],
    "jurisprudence": ['law'],
    "legal_and_moral_basis": ['other'],
    "logical": ['philosophy'],
    "machine_learning": ['computer science'],
    "management": ['business'],
    "marketing": ['business'],
    "marxist_theory": ['philosophy'],
    "modern_chinese": ['linguistics','china specific'],
    "nutrition": ['other'],
    "philosophy": ['philosophy'],
    "professional_accounting": ['business'],
    "professional_law": ['law'],
    "professional_medicine": ['other'],
    "professional_psychology": ['psychology'],
    "public_relations": ['politics'],
    "security_study": ['politics'],
    "sociology": ['culture'],
    "sports_science": ['other'],
    "traditional_chinese_medicine": ['other','china specific'],
    "virology": ['biology'],
    "world_history":['history'],
    "world_religions": ['global'],
}

categories = {
    "STEM": ["physics", "chemistry", "biology", "computer science", "math", "engineering", "statistics"],
    "Humanities": ["history", "philosophy", "law", "arts", "literature", "global"],
    "Social Science": ['linguistics',"business", "politics", "culture", "economics", "geography", "psychology", "education", "sociology"],
    "Other":["other"],
    # "China specific": ["china specific"],
}

In [14]:
mapping = {}

for name, catogories in name_en2zh.items():
    sub_cat = subcategories[name][0]
    for cat, sub_cat_list in categories.items():
        if sub_cat in sub_cat_list:
            category = cat
            break
    mapping[name] = {
        "name": name_en2zh[name],
        "category": cat
    }

In [16]:
import json
with open("/work/u5110390/BenchWeaver/evaluation_data/cmmlu/mapping.json", "w") as f:
    json.dump(mapping, f, ensure_ascii=False, indent=2)

In [21]:
from datasets import load_dataset
import os
import pandas as pd
import zipfile
import shutil

task_list = ['agronomy', 'anatomy', 'ancient_chinese', 'arts', 'astronomy', 'business_ethics', 'chinese_civil_service_exam', 'chinese_driving_rule', 'chinese_food_culture', 'chinese_foreign_policy', 'chinese_history', 'chinese_literature', 
'chinese_teacher_qualification', 'clinical_knowledge', 'college_actuarial_science', 'college_education', 'college_engineering_hydrology', 'college_law', 'college_mathematics', 'college_medical_statistics', 'college_medicine', 'computer_science',
'computer_security', 'conceptual_physics', 'construction_project_management', 'economics', 'education', 'electrical_engineering', 'elementary_chinese', 'elementary_commonsense', 'elementary_information_and_technology', 'elementary_mathematics', 
'ethnology', 'food_science', 'genetics', 'global_facts', 'high_school_biology', 'high_school_chemistry', 'high_school_geography', 'high_school_mathematics', 'high_school_physics', 'high_school_politics', 'human_sexuality',
'international_law', 'journalism', 'jurisprudence', 'legal_and_moral_basis', 'logical', 'machine_learning', 'management', 'marketing', 'marxist_theory', 'modern_chinese', 'nutrition', 'philosophy', 'professional_accounting', 'professional_law', 
'professional_medicine', 'professional_psychology', 'public_relations', 'security_study', 'sociology', 'sports_science', 'traditional_chinese_medicine', 'virology', 'world_history', 'world_religions']


base_dir = "/work/u5110390/BenchWeaver/evaluation_data/cmmlu/data"
for task in task_list:
    dev = load_dataset('haonan-li/cmmlu', task)['dev']
    test = load_dataset('haonan-li/cmmlu', task)['test']
    # Convert datasets to DataFrames
    dev_df = dev.to_pandas()
    dev_df['question'] = dev_df['Question']
    dev_df['answer'] = dev_df['Answer']
    test_df = test.to_pandas()
    test_df['question'] = test_df['Question']
    test_df['answer'] = test_df['Answer']
    # Drop the original columns
    dev_df.drop(columns=['Question', 'Answer'], inplace=True)
    test_df.drop(columns=['Question', 'Answer'], inplace=True)
    # Define file paths
    dev_file = os.path.join(base_dir, 'dev', f"{task}_dev.csv")
    test_file = os.path.join(base_dir, 'test', f"{task}_test.csv")
    # Create directories if they don't exist
    os.makedirs(os.path.dirname(dev_file), exist_ok=True)
    os.makedirs(os.path.dirname(test_file), exist_ok=True)
    # Save DataFrames to CSV files
    dev_df.to_csv(dev_file, index=False)
    test_df.to_csv(test_file, index=False)

# Define the zip file name
zip_file = "cmmlu.zip"

# Function to zip a directory and its contents
def zip_dir(directory, zip_file):
    with zipfile.ZipFile(zip_file, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(directory):
            for file in files:
                zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), directory))

# Zip the directory
zip_dir(base_dir, zip_file)
# Remove the base directory after zipping
shutil.rmtree(base_dir)

Generating test split:   0%|          | 0/105 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/175 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/211 [00:00<?, ? examples/s]

...

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/161 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/160 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

# TAIDE Bench

In [4]:
import json
import pandas as pd
export_dir = "/work/u5110390/BenchWeaver/evaluation_data/taide-bench/data/test"
file_path = "/work/u5110390/BenchWeaver/evaluation_data/taide-bench/data.jsonl"
with open(file_path, "r") as f:
    data = [json.loads(line) for line in f]
dfs = {
    "en2zh": [],
    "zh2en": [],
    "summary": [],
    "letter": [],
    "essay": [],
}
for data_dict in data:
    if data_dict['task'] == "en2zh":
        dfs["en2zh"].append({
            "task": data_dict["task"],
            "qid": data_dict["qid"],
            "question": data_dict["prompt"],
            "answer": data_dict["answer"],
        })
    
for task, df in dfs.items():
    df = pd.DataFrame(df)
    df.to_csv(f"{export_dir}/{task}_test.csv", index=False)
    print(f"Exported {task} data to {export_dir}/{task}_test.csv")

Exported en2zh data to /work/u5110390/BenchWeaver/evaluation_data/taide-bench/data/test/en2zh_test.csv
Exported zh2en data to /work/u5110390/BenchWeaver/evaluation_data/taide-bench/data/test/zh2en_test.csv
Exported summary data to /work/u5110390/BenchWeaver/evaluation_data/taide-bench/data/test/summary_test.csv
Exported letter data to /work/u5110390/BenchWeaver/evaluation_data/taide-bench/data/test/letter_test.csv
Exported essay data to /work/u5110390/BenchWeaver/evaluation_data/taide-bench/data/test/essay_test.csv


# CCPM

In [1]:
import json
train_file_path = "/work/u5110390/BenchWeaver/evaluation_data/ccpm/train.jsonl"
valid_file_path = "/work/u5110390/BenchWeaver/evaluation_data/ccpm/valid.jsonl"
with open(train_file_path, "r") as f:
    train_data = [json.loads(line) for line in f]

with open(valid_file_path, "r") as f: 
    valid_data = [json.loads(line) for line in f]

In [2]:
train_data[0]

{'translation': '诗人啊，你竟像在遥远的地方站立船头。',
 'choices': ['行人初上木兰舟', '骚人遥驻木兰舟', '有人独上木兰舟', '行人迢递木兰舟'],
 'answer': 1}

In [3]:
train_dict = {
    "question": [],
    "A": [],
    "B": [],
    "C": [],
    "D": [],
    "answer": [],
}

valid_dict = {
    "question": [],
    "A": [],
    "B": [],
    "C": [],
    "D": [],
    "answer": [],
}
idx2alphabet = {
    0: "A",
    1: "B",
    2: "C",
    3: "D"
}

In [4]:
for _dict in train_data:
    train_dict['question'].append("以下哪一句古诗最符合现代文「{translation}」的语意？".format(translation=_dict['translation']))
    train_dict['A'].append(_dict['choices'][0])
    train_dict['B'].append(_dict['choices'][1])
    train_dict['C'].append(_dict['choices'][2])
    train_dict['D'].append(_dict['choices'][3])
    train_dict['answer'].append(idx2alphabet[_dict['answer']])

for _dict in valid_data:
    valid_dict['question'].append("以下哪一句古诗最符合现代文「{translation}」的语意？".format(translation=_dict['translation']))
    valid_dict['A'].append(_dict['choices'][0])
    valid_dict['B'].append(_dict['choices'][1])
    valid_dict['C'].append(_dict['choices'][2])
    valid_dict['D'].append(_dict['choices'][3])
    valid_dict['answer'].append(idx2alphabet[_dict['answer']])
    
import pandas as pd    
train_df = pd.DataFrame(train_dict)
valid_df = pd.DataFrame(valid_dict)

In [8]:
train_df.to_csv("/work/u5110390/BenchWeaver/evaluation_data/ccpm/data/dev/all_train.csv", index=False)
valid_df.to_csv("/work/u5110390/BenchWeaver/evaluation_data/ccpm/data/test/all_test.csv", index=False)